# FlyingSquid

In [1]:
#pip install sentence_transformers

In [2]:
from datasets import load_dataset

train = load_dataset("glue", "sst2", split="train")
dev = load_dataset("glue", "sst2", split="validation")

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
import rubrix as rb

records = [
    rb.TextClassificationRecord(
        inputs=text
    )
    for text in train['sentence']
]

records += [
    rb.TextClassificationRecord(
        inputs=text,
        annotation=dev.features['label'].names[label]
    )
    for text, label in zip(dev['sentence'], dev['label'])
]


rb.log(records, name="weak_supervision_sst2")

/home/user/.local/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:18: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


  0%|          | 0/68221 [00:00<?, ?it/s]

68221 records logged to http://localhost:6900/ws/rubrix/weak_supervision_sst2


BulkResponse(dataset='weak_supervision_sst2', processed=68221, failed=0)

In [4]:
from rubrix.labeling.text_classification import Rule, WeakLabels
import rubrix as rb

positive_keywords = [
    "funny", "comedy", "love",
    "fun", "entertaining", "romantic",
    "compelling", "worth", "sweet",
    "fascinating", "laughs", "comic",
    "enjoyable", "clever", "perfect",
    "beautiful", "amusing", "powerful",
    "charming", "engaging", 
]

negative_keywords = [
    "bad", "dull", "worst", "worse",
    "spiritless", "silly", "monotonous", 
    "terrible", "banal", "unimaginative", 
    "inane", "shallow", "offensive", 
    "redundant", "lazy", "loose", 
    "poorly", "awful", "pathetic", 
    "lousy", "inept"
]

rules = [ Rule(query=keyword, label="positive") for keyword in positive_keywords ]
rules += [ Rule(query=keyword, label="negative") for keyword in negative_keywords ]

from rubrix.labeling.text_classification import load_rules

# optionally add the rules defined in the web app UI
rules += load_rules(dataset="weak_supervision_sst2")

# apply the rules to a dataset to obtain the weak labels
weak_labels = WeakLabels(
    rules=rules,
    dataset="weak_supervision_sst2"
)

Preparing rules:   0%|          | 0/41 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/68221 [00:00<?, ?it/s]

In [5]:

from rubrix.labeling.text_classification import FlyingSquid

# we pass our WeakLabels instance to our FlyingSquid label model
flyingsquid_model = FlyingSquid(weak_labels)

# we fit the model
flyingsquid_model.fit()

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(


In [6]:
print(flyingsquid_model.score(tie_break_policy="abstain", output_str=True))

                precision recall f1-score support
efficacy                             0.47     191
fscore_cautious                      0.34     191
coverage                             0.22     191
positive             0.66   0.95     0.78      97
negative             0.90   0.50     0.64      94
accuracy             0.73   0.73     0.73    0.73
macro avg            0.78   0.72     0.71     191
weighted avg         0.78   0.73     0.71     191


In [7]:
import pandas as pd

# get your training records with the predictions of the label model
records_for_training = flyingsquid_model.predict()

# log the records to a new dataset in Rubrix
rb.log(records_for_training, name="flyingsquid_results")

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.inputs["text"], "label": flyingsquid_model.weak_labels.label2int[rec.prediction[0][0]]}
        for rec in records_for_training
    ]
)

  0%|          | 0/9574 [00:00<?, ?it/s]

9574 records logged to http://localhost:6900/ws/rubrix/flyingsquid_results


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=training_data.text.tolist(),
    y=training_data.label.values
)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [9]:
# compute the test accuracy
accuracy = classifier.score(
    X=dev['sentence'],
    y=dev['label']
)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.39908256880733944


# Epoxy

In [10]:
train = load_dataset("glue", "sst2", split="train[:10%]")
dev = load_dataset("glue", "sst2", split="validation")

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [11]:
import rubrix as rb

records = [
    rb.TextClassificationRecord(
        inputs=text
    )
    for text in train['sentence']
]

records += [
    rb.TextClassificationRecord(
        inputs=text,
        annotation=dev.features['label'].names[label]
    )
    for text, label in zip(dev['sentence'], dev['label'])
]


rb.log(records, name="weak_supervision_sst2_epoxy")

  0%|          | 0/7607 [00:00<?, ?it/s]

7607 records logged to http://localhost:6900/ws/rubrix/weak_supervision_sst2_epoxy


BulkResponse(dataset='weak_supervision_sst2_epoxy', processed=7607, failed=0)

In [12]:
# apply the rules to a dataset to obtain the weak labels
weak_labels_epoxy = WeakLabels(
    rules=rules,
    dataset="weak_supervision_sst2_epoxy"
)

Preparing rules:   0%|          | 0/41 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/7607 [00:00<?, ?it/s]

In [13]:
!docker stop $(docker container ls -q)

1c5ea966871d


In [14]:
from sentence_transformers import SentenceTransformer

class SentenceTransformerModel(object):
    
    def __init__(self, embedding_model_name):
        self.embedding_model = SentenceTransformer(embedding_model_name)
    
    def __call__(self, records):
        texts = [ x.inputs["text"] for x in records ]
        embeddings = self.embedding_model.encode(texts)
        return embeddings

In [15]:
from rubrix.labeling.text_classification import Epoxy
embedding_model_name = "average_word_embeddings_glove.840B.300d"
model = SentenceTransformerModel(embedding_model_name)

In [16]:
embeddings = model(weak_labels_epoxy.records())

In [17]:
epoxy = Epoxy(weak_labels_epoxy, embeddings)
epoxy.fit()

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classifica

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

...

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/rubrix/lib/python3.8/site-packages/pgmpy/models/MarkovModel.py:8: FutureWarning: MarkovModel has been renamed to MarkovNetwork. Please use MarkovNetwork class, MarkovModel will be removed in future.
  warnings.warn(
/home/user/miniconda3/envs/r

In [19]:
print(epoxy.score(tie_break_policy="abstain"))

{'efficacy': 0.7730520441381643, 'fscore_cautious': 0.7090921520371889, 'coverage': 0.9954128440366973, 'positive': {'precision': 0.5326370757180157, 'recall': 0.9272727272727272, 'f1-score': 0.6766169154228855, 'support': 440}, 'negative': {'precision': 0.6862745098039216, 'recall': 0.16355140186915887, 'f1-score': 0.2641509433962264, 'support': 428}, 'accuracy': 0.5506912442396313, 'macro avg': {'precision': 0.6094557927609686, 'recall': 0.545412064570943, 'f1-score': 0.470383929409556, 'support': 868}, 'weighted avg': {'precision': 0.6083937828479324, 'recall': 0.5506912442396313, 'f1-score': 0.4732350766816296, 'support': 868}}


In [21]:
# get your training records with the predictions of the label model
records_for_training = epoxy.predict()

# log the records to a new dataset in Rubrix

# extract training data
training_data = pd.DataFrame(
    [
        {"text": rec.inputs["text"], "label": epoxy.weak_labels.label2int[rec.prediction[0][0]]}
        for rec in records_for_training
    ]
)

In [22]:
# preview training data
training_data

,text,label
0,"the better video-game-based flicks ,",1
1,bungle their way,0
2,in a standard plot,1
3,not everything in this ambitious comic escapad...,1
4,tom shadyac 's film kicks off spookily enough,0
...,...,...
6693,will turn out okay,1
6694,solid base,0
6695,"violent , vulgar and forgettably entertaining .",1
6696,"its exquisite acting , inventive screenplay , ...",1


In [23]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=training_data.text.tolist(),
    y=training_data.label.values
)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [24]:
# compute the test accuracy
accuracy = classifier.score(
    X=dev['sentence'],
    y=dev['label']
)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.5011467889908257
